In [1]:
import torch
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

import sys
sys.path.append('../')

import numpy as np
from torch.utils.data import Dataset
from transformers import BertTokenizer, AutoTokenizer
from tqdm import tqdm
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.classes.preprocessor import TextPreProcessor
from model1.spanemo.learner import Trainer, EvaluateOnTest
from model1.spanemo.model import SpanEmo
from model1.spanemo.data_loader import DataClass
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import datetime
import numpy as np
import json

c:\Users\tianq\Documents\Deep learning\deepEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
utt_list = []
convo_list = []
i = 1


j = 0
for dirpath, dirnames, filenames in os.walk("./data/iemocap/raw"):
    if i == 1:
        i += 1
    else:
        print('========= ', dirpath)
        for file in listdir(dirpath):
            print(file)
            raw = open(dirpath + "/"+ file, "r")
            
            for a in raw:
                index = a.index(':')
                utt_list.append(a[index+2:-1])
                convo_list.append(j)
            j += 1
            
        


=========  ./data/iemocap/raw\session1
Ses01F_impro01.txt
Ses01F_impro02.txt
Ses01F_impro03.txt
Ses01F_impro04.txt
Ses01F_impro05.txt
Ses01F_impro06.txt
Ses01F_impro07.txt
Ses01F_script01_1.txt
Ses01F_script01_2.txt
Ses01F_script01_3.txt
Ses01F_script02_1.txt
Ses01F_script02_2.txt
Ses01F_script03_1.txt
Ses01F_script03_2.txt
Ses01M_impro01.txt
Ses01M_impro02.txt
Ses01M_impro03.txt
Ses01M_impro04.txt
Ses01M_impro05.txt
Ses01M_impro06.txt
Ses01M_impro07.txt
Ses01M_script01_1.txt
Ses01M_script01_2.txt
Ses01M_script01_3.txt
Ses01M_script02_1.txt
Ses01M_script02_2.txt
Ses01M_script03_1.txt
Ses01M_script03_2.txt
=========  ./data/iemocap/raw\session2
Ses02F_impro01.txt
Ses02F_impro02.txt
Ses02F_impro03.txt
Ses02F_impro04.txt
Ses02F_impro05.txt
Ses02F_impro06.txt
Ses02F_impro07.txt
Ses02F_impro08.txt
Ses02F_script01_1.txt
Ses02F_script01_2.txt
Ses02F_script01_3.txt
Ses02F_script02_1.txt
Ses02F_script02_2.txt
Ses02F_script03_1.txt
Ses02F_script03_2.txt
Ses02M_impro01.txt
Ses02M_impro02.txt
Ses0

In [6]:
utt_list

['Excuse me.',
 'Do you have your forms?',
 'Yeah.',
 'Let me see them.',
 'Is there a problem?',
 'Who told you to get in this line?',
 'You did.',
 'You were standing at the beginning and you directed me.',
 "Okay. But I didn't tell you to get in this line if you are filling out this particular form.",
 "Well what's the problem?  Let me change it.",
 'This form is a Z.X.four.',
 "You can't--  This is not the line for Z.X.four.  If you're going to fill out the Z.X.four, you need to have a different form of ID.",
 "What?  I'm getting an ID.  This is why I'm here.  My wallet was stolen.",
 'No. I need another set of ID to prove this is actually you.',
 'How am I supposed to get an ID without an ID?  How does a person get an ID in the first place?',
 "I don't know.  But I need an ID to pass this form along.  I can't just send it along without an ID.",
 "I'm here to get an ID.",
 'No.  I need another ID, a separate one.',
 'Like what?  Like a birth certificate?',
 "A birth certificate, a 

In [7]:
convo_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [8]:
utt_df = pd.DataFrame(columns=['transcription'])
utt_df['transcription'] = utt_list
utt_df['conversation_no'] = convo_list

In [9]:
utt_df.to_csv('./data/iemocap/preprocessed_iemo.csv', index=False)

In [10]:
utt_df = pd.read_csv('./data/iemocap/preprocessed_iemo.csv')

In [11]:
utt_df

,transcription,conversation_no
0,Excuse me.,0
1,Do you have your forms?,0
2,Yeah.,0
3,Let me see them.,0
4,Is there a problem?,0
...,...,...
10234,oh! Marry you again? I wouldn't marry you agai...,150
10235,Beast,150
10236,You're a wicked little vampire. And I pray to...,150
10237,Brute,150


In [17]:
utt_text = np.array(utt_df['transcription'])

In [ ]:
hyperparams = {
    'train_path':utt_text, 
    'backbone':'bert-base-uncased',
    'max_length': 128, #maximum context length for the model
   
}

pandas.core.series.Series

In [ ]:
train_dataset = DataClass(hyperparams, utt_text, pred_mode=True)
train_data_loader = DataLoader(train_dataset,
                                    batch_size=128,
                                    shuffle=False)

In [ ]:
def test(model, model_path, loader=None):
    test_data_loader = loader
    
    learn = EvaluateOnTest(model, test_data_loader, model_path)
    learn.predict(device=device)

In [ ]:
test(model, model_path, train_data_loader)